In [1]:
import os

In [2]:
%cd C:\Users\harsh\Downloads\ml_project


C:\Users\harsh\Downloads\ml_project


C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'C:\\Users\\harsh\\Downloads\\ml_project'

In [4]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/harshita0212/OenoFlow-smart-wine-prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "harshita0212"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "a96f4207556acabc9f145184bde3390ce52c2f88"


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
import sys
import os

sys.path.append(os.path.abspath("src"))
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/harshita0212/OenoFlow-smart-wine-prediction.mlflow",
           
        )

        return model_evaluation_config


In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import joblib
import json

from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Helper to save scores locally
def save_json(path: Path, data: dict):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted)
            #RMSE: How far predictions are from actual values (in same unit as target).
            #MAE: Average absolute error.
            #R² Score: How well your model explains the variance in the data (closer to 1 = better).

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(Path(self.config.metric_file_name), scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # 💡 Save model manually & log as artifact
            model_path = Path("models/sk_model.pkl")
            model_path.parent.mkdir(parents=True, exist_ok=True)
            joblib.dump(model, model_path)
            mlflow.log_artifact(str(model_path))


In [15]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    evaluator = ModelEvaluation(config=model_eval_config)
    evaluator.log_into_mlflow()
except Exception as e:
    raise e


[2025-07-22 11:49:23,327: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-22 11:49:23,329: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-22 11:49:23,332: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-22 11:49:23,334: INFO: common: created directory at: artifacts]
[2025-07-22 11:49:23,334: INFO: common: created directory at: artifacts/model_evaluation]


c:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


🏃 View run respected-mole-81 at: https://dagshub.com/harshita0212/OenoFlow-smart-wine-prediction.mlflow/#/experiments/0/runs/c83dc08e0c2b4747887f6f6a8b1c88e1
🧪 View experiment at: https://dagshub.com/harshita0212/OenoFlow-smart-wine-prediction.mlflow/#/experiments/0
